In [ ]:
!pip install transformers==4.21.2
!pip install tokenizers==0.12.1
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.8 MB/s 
     |████████████████████████████████| 182 kB 34.3 MB/s 
     |████████████████████████████████| 6.6 MB 53.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 6.6 MB/s 


**Config**

In [ ]:
class CFG:
  num_workers=2
  path="/content/drive/MyDrive/Colab Notebooks/Erdos Fall 2022/" # path of the project
  config_path=path+'config.pth'
  model="microsoft/deberta-v3-base"
  gradient_checkpointing=False
  batch_size=10
  target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
  seed=42
  n_fold=3
  trn_fold=range(n_fold)

**Libraries**

In [ ]:
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold, train_test_split

os.system('pip install iterative-stratification==0.1.7')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

import tokenizers
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import DataCollatorWithPadding
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

env: TOKENIZERS_PARALLELISM=true


**Tokenizer**

In [ ]:
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')

**Utils**

In [ ]:
# ====================================================
# Utils
# ====================================================
def MCRMSE(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]
        y_pred = y_preds[:,i]
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores


def get_score(y_trues, y_preds):
    mcrmse_score, scores = MCRMSE(y_trues, y_preds)
    return mcrmse_score, scores


def get_logger(filename='inference'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

**OOF**

In [ ]:
oof_df = pd.read_pickle(CFG.path+'oof_df.pkl')
labels = oof_df[CFG.target_cols].values
preds = oof_df[[f"pred_{c}" for c in CFG.target_cols]].values
score, scores = get_score(labels, preds)
LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')

Score: 0.4567  Scores: [0.49493978380259734, 0.4482033941930121, 0.41435760596752336, 0.460858547153225, 0.4764635149720428, 0.44535385542533723]
INFO:__main__:Score: 0.4567  Scores: [0.49493978380259734, 0.4482033941930121, 0.41435760596752336, 0.460858547153225, 0.4764635149720428, 0.44535385542533723]


**Data Loading**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_new.csv')
train, test = train_test_split(df, test_size=0.2, shuffle = True, random_state = 440)
train=train.reset_index(drop=True)
test= test.reset_index(drop=True)

print(f"df.shape: {df.shape}")
display(df.head())

print(f"train.shape: {train.shape}")
display(train.head())

print(f"test.shape: {test.shape}")
display(test.head())

df.shape: (3945, 8)


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


train.shape: (3156, 8)


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,4D23A7FA9408,Although some say students wouldn't benefit at...,2.5,2.5,3.0,2.5,2.5,3.5
1,57DFE525E50A,Ever since I was in elementary school I have h...,4.0,4.5,5.0,4.5,4.5,4.0
2,EC07B11B2CF3,a positive attitude is the key to success in l...,2.0,2.0,3.0,2.5,2.0,2.0
3,B5FFED1536FE,I disagree with Emerson because if they have a...,2.5,2.0,3.0,2.0,2.5,3.5
4,845693DC199E,"Dear principal, ''This is what i think about c...",4.0,3.5,4.0,3.0,4.0,4.0


test.shape: (789, 8)


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0C28481DFBB8,"In my opinion, I know why Thomas Jefferson dec...",4.0,3.0,3.0,3.0,2.5,3.5
1,05E5653B781C,its always good to ask people how it feels or ...,3.0,3.5,3.0,3.0,3.5,3.0
2,F075F9AD2AA4,Do you think it is a good idea for the student...,2.5,2.5,2.5,2.0,2.0,2.0
3,ACA1A45EE438,I strongly disagree with extending the school ...,4.0,3.0,3.0,4.0,4.0,4.0
4,A0D47C0DD67F,Individuality is the idea of freedom of though...,3.0,3.0,3.0,2.0,2.0,2.0


In [ ]:
# sort by length to speed up inference
test['tokenize_length'] = [len(CFG.tokenizer(text)['input_ids']) for text in test['full_text'].values]
test = test.sort_values('tokenize_length', ascending=True).reset_index(drop=True)
display(test.head())

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,tokenize_length
0,5FFB959ED7BA,"I agree.\n\nI agree with this statement, becau...",2.5,2.5,2.5,2.5,3.0,3.0,61
1,0DB44DDF91E6,Dear principal.\r\n\r\nWe like to clean litter...,3.0,3.0,3.0,2.5,3.0,3.0,62
2,E80B0AD2A9D7,I think is important because de Churchill have...,1.5,2.0,2.0,2.0,2.0,2.0,118
3,B0DFC5E93591,First i think what if someone have a agree and...,2.0,2.0,2.0,2.0,2.0,2.5,123
4,53E542321463,"Dear Principal,\n\nI agree with what you think...",3.0,3.0,3.0,2.5,3.0,4.0,146


**DataSet**

In [ ]:
def prepare_input(cfg, text):
    inputs = cfg.tokenizer.encode_plus(
        text, 
        return_tensors=None, 
        add_special_tokens=True, 
        #max_length=CFG.max_len,
        #pad_to_max_length=True,
        #truncation=True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['full_text'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        return inputs

**Model**

In [ ]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings
    

class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()
        self.pool = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, 6)
        self._init_weights(self.fc)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        feature = self.pool(last_hidden_states, inputs['attention_mask'])
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(feature)
        return output

**Evaluation**

In [ ]:
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [ ]:
test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         collate_fn=DataCollatorWithPadding(tokenizer=CFG.tokenizer, padding='longest'),
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    with open(CFG.path+"pred_{}".format(fold), "wb") as fp:   #Pickling
      pickle.dump(prediction, fp)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
#predictions = np.mean(predictions, axis=0)

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/Erdos Fall 2022/pred_0", "rb") as fp:   # Unpickling
  pred_0 = pickle.load(fp)
with open("/content/drive/MyDrive/Colab Notebooks/Erdos Fall 2022/pred_1", "rb") as fp:   # Unpickling
  pred_1 = pickle.load(fp)
with open("/content/drive/MyDrive/Colab Notebooks/Erdos Fall 2022/pred_2", "rb") as fp:   # Unpickling
  pred_2 = pickle.load(fp)

In [ ]:
predictions.append(pred_0)
predictions.append(pred_1)
predictions.append(pred_2)

In [ ]:
predictions = np.mean(predictions, axis=0)

In [ ]:
predictions

array([[2.47235  , 2.434372 , 2.6094685, 2.4379892, 2.5645607, 2.7292547],
       [2.507916 , 2.559075 , 2.6231344, 2.519471 , 2.8750865, 2.89758  ],
       [2.261851 , 2.0704079, 2.493509 , 2.0639257, 2.0209196, 2.45379  ],
       ...,
       [4.8698583, 4.9461656, 4.9310308, 4.915361 , 4.9950805, 4.918737 ],
       [4.949182 , 4.985867 , 4.9775963, 4.959343 , 4.9504986, 4.954241 ],
       [4.912668 , 4.984127 , 4.9655933, 4.941535 , 4.973755 , 4.928446 ]],
      dtype=float32)

In [ ]:
target_values=test.drop(columns=['text_id','full_text','tokenize_length']).values

In [ ]:
mses = ((target_values-predictions)**2).mean(axis=0)

In [ ]:
mses

array([0.23839074, 0.19228072, 0.16764401, 0.18373389, 0.19578173,
       0.20649405])

In [ ]:
rmses=np.sqrt(mses)
rmses

array([0.48825274, 0.43849825, 0.40944354, 0.42864191, 0.44247229,
       0.45441617])

In [ ]:
score= np.mean(rmses)
score

0.44362081672191994